In [1]:
%matplotlib inline
import datetime
import calendar
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = [15, 6]
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
data = pd.read_csv('data/raw.csv', encoding='utf-8')
data.head()

,Date,Day of Week,# in Line,Line-up Time,Pick-up Time,Notes
0,6/8/2016,Wednesday,19,7:50:00 AM,8:05:00 AM,NaN
1,6/9/2016,Thursday,4,7:26:00 AM,7:29:00 AM,NaN
2,6/9/2016,Thursday,19,8:01:00 AM,8:21:00 AM,Bus arrived as I got in line
3,6/10/2016,Friday,9,7:30:00 AM,7:38:00 AM,NaN
4,6/10/2016,Friday,21,7:50:00 AM,8:00:00 AM,Took the bus after 10 minutes


In [3]:
data['timestamp_arrive'] = pd.to_datetime(data['Date']+' '+data['Line-up Time'])

In [4]:
data['timestamp_depart'] = pd.to_datetime(data['Date']+' '+data['Pick-up Time'])

In [6]:
data = data.rename(columns={'Day of Week':'weekday',
                            '# in Line':'line_count',
                            'Notes':'notes'})

In [7]:
data = data.drop(['Date', 'Line-up Time', 'Pick-up Time'], axis=1)

In [8]:
data.head()

,weekday,line_count,notes,timestamp_arrive,timestamp_depart
0,Wednesday,19,NaN,2016-06-08 07:50:00,2016-06-08 08:05:00
1,Thursday,4,NaN,2016-06-09 07:26:00,2016-06-09 07:29:00
2,Thursday,19,Bus arrived as I got in line,2016-06-09 08:01:00,2016-06-09 08:21:00
3,Friday,9,NaN,2016-06-10 07:30:00,2016-06-10 07:38:00
4,Friday,21,Took the bus after 10 minutes,2016-06-10 07:50:00,2016-06-10 08:00:00


In [ ]:
df = data[['line_count', 'timestamp_arrive']]
df['timestamp'] = df['timestamp_arrive'].apply(lambda x: x.replace(month=1, day=1, year=2017))
df = df.set_index('timestamp')
df = df.sort_index()
df = df.drop('timestamp_arrive', axis=1)
df.head()

In [ ]:
grp = df.groupby(pd.TimeGrouper(freq='10min'))
aggdf = grp.agg(['mean', 'count'])
aggdf.index = aggdf.index.time
aggdf.plot(kind='bar', subplots=True)
# plt.savefig('output.png')

In [ ]:
aggdf.head()

In [ ]:
def plt_wkday(weekday):
    df = data[data['weekday'] == weekday][['line_count', 'timestamp_arrive']]
    df['timestamp'] = df['timestamp_arrive'].apply(lambda x: x.replace(month=1, day=1, year=2017))
    df = df.set_index('timestamp')
    df = df.sort_index()
    df = df.drop('timestamp_arrive', axis=1)
    grp = df.groupby(pd.TimeGrouper(freq='10min'))
    aggdf = grp.mean()
    aggdf.index = aggdf.index.time
#     aggdf.plot()
#     plt.savefig('weekday_chart.png')
    print(weekday+' Average')
    print(aggdf)

In [ ]:
weekday = calendar.day_name[date.today().weekday()]

if weekday == 'Saturday' or weekday == 'Sunday':
    weekday = 'Monday'

print(weekday)

In [ ]:
plt_wkday(weekday)